# Efficient Deep Learning using Transfer Learning


## Transfer Learning 

### Deep Learning as Function Approximation

Last time we introduced functions which is a very general concept and can help us solve a number of problems. In this world, we look at our data as input(x) and output(y) and try to find a mapping from x to y.


|Scenario|Data=(x,y)|Function f s.t. y=f(x)|
|---|---|---|
| Best fit line| 2D points(x,y)  | y=f(x)  |
| Image classification | pixels representing picture  | image-class = f(image)   |
| Audio2Text  | wave representing utterances  | words = f(audio-wave)  |
| Translation  | sentences in multiple languages  | french-sentence = f(english-sentence)  |
 
 We can write functions in 2 ways.
 * Manually : Humans write the logic in function body.
 * ML way   : We provide data to a Deep Learning machine and it writes the function for us.

\

\

 ![Dl-Model](https://drive.google.com/uc?export=view&id=1Tiwy1JJdP3Mu6URLVn1xjL0r8JQ0hbcv)

### Questions
* What do functions learn?
* How do we choose architecture for learning?
* How can we efficiently learn?

### Short Answers
* Functions learn weights. For e.g. in case of best fit line, we need to find y=f(x). We know a line is represented as y=mx+c . So our weights are w=[m,c]
* Architecture is chosen based on the domain. For e.g. vision problems do well with CNN based models like ResNet. Transformer models are used heavily in text domain.
* Efficient learning can be achieved by function reuse, not unlike the libraries we use in our software development. 

In fact, understanding what functions learn gives us great insights into **efficient learning**. 
* Efficiency in compute 
* Efficiency in dev time

### Transfer Learning - Motivation 

Zeiler and Fergus looked at various layers and found a way to visualize weights and activations to understand what the cnn model is learning.
We now look at a trained neural network on image classification task. Instead of looking at weights as a collection of numbers, we visualize these by looking at feature activation and corresponding images.


* Layer 1-2
![Layer1-2](https://drive.google.com/uc?export=view&id=15PQuA7c0uqcPb_CmpcWpT38gT8QxRbNl)


* Layer 3
![Layer3](https://drive.google.com/uc?export=view&id=1wDWyvPs5aHOdcxJDUw-dzsPEQhJSFQGU)


* Layer 4-5
![Layer4-5](https://drive.google.com/uc?export=view&id=1PWDZoUEBaHCkvq7vTCxw1Z06dqzqFxBq)


\
\
We realize that hidden layers progressively capture complex features as we move from input to output. This also means that earlier layers can be useful in many different tasks. For e.g. a model trained on image classification can be re-used for digit(1-9) recoginition. This gives us the intuition to **Transfer Learning**.



### Transfer Learning - Concept
A model is composed of body & head. We need to take the body of pre-trained model & attach a new head for our specific task.

![Transfer Learning](https://drive.google.com/uc?export=view&id=1XTfY6_piQdzEm0nmhec6uKZMMTitIVOf)

* Pull a pre-trained model from repository.
* Remove the head and attach a new head to the model body.
* Use task specific dataset and use it to train only the new head.
* Unlock the pretrained body and train the complete model. This is called fine-tuning.

Transfer learning should remind us of code reuse. We get efficient by reusing libraries. Another intuition from code-refactoring is that later layers will change more for a novel task. This intuition is exploited via discriminative learning rate where we learn slower for earlier layers. 

Before we move forward, let us look at higher level deep learning libray **FastAI**


## FastAI
[FastAI](https://github.com/fastai/fastai) is a high level deep learning library that allows us to solve real life problems efficiently. It is built on top of PyTorch. 

### FastAI - Layered API
Fast AI offers a layered api that provides friendly abstractions for common use cases through high-level api. It also provides mid-level and low-level apis that give us great control for more complex and non-standard use cases.

![FastAI Layered Api](https://drive.google.com/uc?export=view&id=13pZS238UHmqYPP_iA41J1ZscVXMx8fEl)



### FastAI - Object model for High level API

Training in deep learning requires following high level components.
* **DataLoader** : How to load data from files and send it in a format that deep learning machine can consume?
* **Model** : Provides a framework or architecture. Once learning is complete, we say that the model has learnt for the task.
* **Loss** : Loss is a way for evaluating the quality of model. This acts as a feedback system to progressively improve the model.

FastAI's *Learner* abstraction brings these components together. Let us look at Learner and DataLoader.

\

\

![FastAI Object Model for High Level API](https://drive.google.com/uc?export=view&id=1FYT5fA1uRaFmsq5K8QFyGA4yoeusDiMI)




### Transfer Learning Example : Sentiment Analysis using FastAI

Transfer learning works very well both on vision an textual tasks. 
We will do sentiment analysis on text data using Transfer Learning. We will use FastAI which is built on top of PyTorch. Our pre-trained model is AWD_LSTM.


In [2]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 6.8 MB/s 
     |████████████████████████████████| 4.7 MB 58.5 MB/s 
     |████████████████████████████████| 365 kB 68.1 MB/s 
     |████████████████████████████████| 1.2 MB 62.1 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 115 kB 78.8 MB/s 
     |████████████████████████████████| 212 kB 75.7 MB/s 
     |████████████████████████████████| 141 kB 73.9 MB/s 
     |████████████████████████████████| 596 kB 72.5 MB/s 
     |████████████████████████████████| 127 kB 68.9 MB/s 
     |████████████████████████████████| 6.6 MB 51.5 MB/s 
Mounted at /content/gdrive


In [3]:
#hide
from fastbook import *

In [4]:
#Load IMDB dataset (movie_review -> sentiment)
from fastai.text.all import *
path = untar_data(URLs.IMDB)
dls = TextDataLoaders.from_folder(path, valid='test')
path_str = path.as_posix()

path

Path('/root/.fastai/data/imdb')

In [9]:
#Sentiment analysis movie review data, source & destination
print("Data downloaded from {0} to {1}".format(URLs.IMDB, path_str))

Data downloaded from https://s3.amazonaws.com/fast-ai-nlp/imdb.tgz to /root/.fastai/data/imdb


In [10]:
#Destination folders
!ls $path_str

imdb.vocab  README  test  tmp_clas  tmp_lm  train  unsup


In [11]:
#review sample text
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of oatmeal . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain overconfidence on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an idyllic storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",neg
3,"xxbos i have never seen any of xxmaj spike xxmaj lee 's prior films , as their trailers never caught my interest . i have seen , and admire xxmaj denzel xxmaj washington , and xxmaj jodie xxmaj foster 's work , and have several of their dvds . i was , however , entirely disappointed with this movie . xxmaj if this film is any indication of xxmaj spike xxmaj lee 's ability as a director , my advice would be to "" get a job "" , and stop wasting the time and talent of others . \n\n i wonder if some of the other imdb commentators watched the same movie that xxmaj i 'd seen . i can only assume , from their sappy lovelorn reviews , that their adoration of xxmaj spike xxmaj lee has blinded them to the banality of this piece of work .",neg
4,"xxbos i felt duty bound to watch the 1983 xxmaj timothy xxmaj dalton / xxmaj zelah xxmaj clarke adaptation of "" jane xxmaj eyre , "" because xxmaj i 'd just written an article about the 2006 xxup bbc "" jane xxmaj eyre "" for xxunk . \n\n xxmaj so , i approached watching this the way xxmaj i 'd approach doing homework . \n\n i was irritated at first . xxmaj the lighting in this version is bad . xxmaj everyone / everything is washed out in a bright white xxunk light that , in some scenes , casts shadows on the wall behind the characters . \n\n xxmaj and the sound is poorly recorded . i felt like i was listening to a high school play . \n\n xxmaj and the pancake make - up is way too heavy . \n\n xxmaj and the sets do n't fully",pos
5,"xxbos xxmaj pier xxmaj paolo xxmaj pasolini , or xxmaj pee - pee - pee as i prefer to 

In [12]:
#define learner with data-loader & pre-trained text model
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [14]:
#language model location
!ls /root/.fastai/models/wt103-fwd

itos_wt103.pkl	lstm_fwd.pth


In [15]:
#fine tune pre-trained model only 1 epoch in the interest of time
learn.fine_tune(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.455680,0.389491,0.824520,02:22


epoch,train_loss,valid_loss,accuracy,time
0,0.279893,0.229843,0.908480,04:11


In [16]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos xxmaj if anyone ever assembles a compendium on modern xxmaj american horror that is truly worth it 's salt , there will * have * to be an entry for xxup sf xxmaj brownrigg 's xxunk xxunk in xxmaj asylum xxmaj horror . xxmaj every time i watch this movie i am impressed by the complete economy of the film , from the compact , totally self - contained plot with a puzzling beginning and an all too horrible ending , the engaging performances by what was essentially a group of non - professional actors , and a xxunk sense of dread and claustrophobia that effectively consumes the xxunk with a certain inevitability which is all the more terrifying because the viewers know what is going on long before the xxunk ] , with the only question being when are they going to wake up & smell the coffee",pos,pos
2,"xxbos xxmaj years ago , when xxup darling xxup lili played on xxup tv , it was always the pan and scan version , which i hated and decided to wait and see the film in its proper widescreen format . xxmaj so when i saw an inexpensive xxup dvd of this xxmaj julie xxmaj andrews / xxmaj blake xxmaj edwards opus , i decided to purchase and watch it once and for all . \n\n xxmaj boy , what a terrible film . xxmaj it 's so bad and on so many levels that i really do not know where to start in describing where and when it goes so horribly wrong . xxmaj looking at it now , it 's obvious to any fans of movies that xxmaj blake xxmaj edwards created this star vehicle for his wife simply because so many other directors had struck gold with",neg,neg
3,"xxbos xxmaj if you 've seen the trailer for this movie , you pretty much know what to expect , because what you see here is what you get . xxmaj and even if you have n't seen the previews , it wo n't take you long to pick up on what you 're in for-- specifically , a good time and plenty of xxunk from this clever satire of ` reality xxup tv ' shows and ` buddy xxmaj cop ' movies , ` showtime , ' directed by xxmaj tom xxmaj dey , starring xxmaj robert xxmaj de xxmaj niro and xxmaj eddie xxmaj murphy . \n\n\t xxmaj mitch xxmaj preston ( de xxmaj niro ) is a detective with the xxup l.a.p.d . , and he 's good at what he does ; but working a case one night , things suddenly go south when another cop",pos,pos
4,"xxbos "" oh , you pilots are such men . "" "" they do n't call it the cockpit for nothing , honey . "" xxmaj dialogue like that is just one of many reasons why xxmaj the xxmaj concorde ▁ xxmaj airport ' 79 ( or , if you saw it in the xxup uk where it dragged its heels getting released there , xxmaj airport ' 80 : xxmaj the xxmaj concorde ) was the last and by far the least of the series . xxmaj the disaster movie was in dire straits in the xxunk , what with xxmaj the xxmaj swarm having offered much unintentional hilarity and xxmaj beyond the xxmaj poseidon xxmaj adventure , xxmaj when xxmaj time xxmaj ran xxmaj out and xxmaj city on xxmaj fire simply offering much boredom , and the desperation to find a new spin on the genre is",neg,neg
5,"xxbos i saw this on sale - xxup new - at my local store for $ 6 and said "" hey ! an action film with that guy from xxmaj bloodsport and xxmaj enter the xxmaj dragon , directed by the guy who did xxmaj enter the xxmaj